# Lab 2 - Automatic Creation of `sqlalchemy` data type `dict`

So far, we have been manually constructing the `sqlalchemy` type `dict`, but this approach quickly becomes unwieldy.  Let's look at using the `pandas` types to programmically construct the type `dict`.

In [2]:
import pandas as pd
from dfply import *

In [3]:
!rm databases/baseball.db

rm: databases/baseball.db: No such file or directory


## Case Study - People

Let's use the `People.csv` file from the [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/) as our motivating example, since 

1. It has examples of a number of types
2. It has lots of columns and would be annoying to manually construct the type `dict`.

In [4]:
people = pd.read_csv('~/Desktop/baseball/core/People.csv')
people.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [5]:
people.shape

(19370, 24)

## <font color="red"> Exercise 1 </font>

Inspect the types of the `people` table and make note of any necessary changes.

In [9]:
# Your code here
people.dtypes

playerID         object
birthYear       float64
birthMonth      float64
birthDay        float64
birthCountry     object
birthState       object
birthCity        object
deathYear       float64
deathMonth      float64
deathDay        float64
deathCountry     object
deathState       object
deathCity        object
nameFirst        object
nameLast         object
nameGiven        object
weight          float64
height          float64
bats             object
throws           object
debut            object
finalGame        object
retroID          object
bbrefID          object
dtype: object

*Your thoughts here*
year, month, and day should be int,
change debut and finalGame to date-time

## Missing `Int64` columns

As mentioned in [Lecture 1/4](./pbpython/notebooks/1_4_more_on_pandas_data_types_key.ipynb), we need to use the most recent version `pandas` (still in development as of this writing) to allow us to have integer columns with missing values.

In [16]:
assert pd.__version__.startswith('0.24'), "Please uncomment and run the pip command to upgrade pandas"
#!pip install --upgrade --pre pandas

## Correcting the `pandas` types

1. We pass `parse_dates` a list of date columns
2. We pass `dtypes` a `dict` of types of the birth and death columns

#### Constructing the `dtype` `dict`

In [17]:
date_cols = ['debut', 'finalGame']
date_cols

['debut', 'finalGame']

In [18]:
birth_death_date_cols = [prefix + time for prefix in ('birth', 'death') for time in ('Year', 'Month', 'Day')]
birth_death_date_cols

['birthYear', 'birthMonth', 'birthDay', 'deathYear', 'deathMonth', 'deathDay']

In [19]:
people_dtypes = {col:pd.Int64Dtype() for col in people.columns if col in birth_death_date_cols}
people_dtypes

{'birthDay': Int64Dtype(),
 'birthMonth': Int64Dtype(),
 'birthYear': Int64Dtype(),
 'deathDay': Int64Dtype(),
 'deathMonth': Int64Dtype(),
 'deathYear': Int64Dtype()}

## Rereading the csv with the correct types

In [21]:
people = pd.read_csv('~/Desktop/baseball/core/People.csv', dtype=people_dtypes, parse_dates=date_cols)
people.dtypes

playerID                object
birthYear                Int64
birthMonth               Int64
birthDay                 Int64
birthCountry            object
birthState              object
birthCity               object
deathYear                Int64
deathMonth               Int64
deathDay                 Int64
deathCountry            object
deathState              object
deathCity               object
nameFirst               object
nameLast                object
nameGiven               object
weight                 float64
height                 float64
bats                    object
throws                  object
debut           datetime64[ns]
finalGame       datetime64[ns]
retroID                 object
bbrefID                 object
dtype: object

In [22]:
people.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981,12,27,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934,2,5,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954,9,8,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972,8,25,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


## <font color="red"> Exercise 2 </font>
**Goal:** Find a method/attribute of each `dtype`, which preferably returns something immutable like a `str`, that we can use to identify the general type.
**Tasks:**

1. Pull off an example `dtype`
2. Use `dir` to inspect the available methods
3. Test the methods/attributes to find a good candidate.

In [23]:
# Your code here
dir(people.birthDay.dtype)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_is_boolean',
 '_is_numeric',
 '_metadata',
 'base',
 'construct_array_type',
 'construct_from_string',
 'is_dtype',
 'is_signed_integer',
 'is_unsigned_integer',
 'itemsize',
 'kind',
 'na_value',
 'name',
 'names',
 'numpy_dtype',
 'type']

In [40]:
from toolz import first
example_type = first(people.dtypes)
example_type

dtype('O')

In [42]:
[m for m in dir(example_type) if not m.startswith('_')]

['alignment',
 'base',
 'byteorder',
 'char',
 'descr',
 'fields',
 'flags',
 'hasobject',
 'isalignedstruct',
 'isbuiltin',
 'isnative',
 'itemsize',
 'kind',
 'metadata',
 'name',
 'names',
 'ndim',
 'newbyteorder',
 'num',
 'shape',
 'str',
 'subdtype',
 'type']

In [48]:
[t.kind for t in people.dtypes]

['O',
 'i',
 'i',
 'i',
 'O',
 'O',
 'O',
 'i',
 'i',
 'i',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'f',
 'f',
 'O',
 'O',
 'M',
 'M',
 'O',
 'O']

## Creating a type conversion dictionary

1. keys will be the `dtype.kind` strings
2. The `values` will be the associated `sqlalchemy` types

In [49]:
from sqlalchemy import Integer, Float, String, DateTime
DTYPES_TO_SQLALCHEMY_TYPES = {'O':String,
                              'i':Integer,
                              'f':Float,
                              'M':DateTime}
DTYPES_TO_SQLALCHEMY_TYPES

{'M': sqlalchemy.sql.sqltypes.DateTime,
 'O': sqlalchemy.sql.sqltypes.String,
 'f': sqlalchemy.sql.sqltypes.Float,
 'i': sqlalchemy.sql.sqltypes.Integer}

## Use ALL CAPS for global constants

When dealing with global constants, we should

1. Define them at the top of the file.
2. Use an ALL CAPS name to make them stand out.

## <font color="red"> Exercise 3 </font>

Write a `dict` comprehension that uses our conversion `dict` to convert the `pandas` `dtypes` to `sqlalchemy` types.

In [58]:
# Your code here
{col:DTYPES_TO_SQLALCHEMY_TYPES[dtype.kind] for dtype, col in zip(people.dtypes, people.columns)}

{'bats': sqlalchemy.sql.sqltypes.String,
 'bbrefID': sqlalchemy.sql.sqltypes.String,
 'birthCity': sqlalchemy.sql.sqltypes.String,
 'birthCountry': sqlalchemy.sql.sqltypes.String,
 'birthDay': sqlalchemy.sql.sqltypes.Integer,
 'birthMonth': sqlalchemy.sql.sqltypes.Integer,
 'birthState': sqlalchemy.sql.sqltypes.String,
 'birthYear': sqlalchemy.sql.sqltypes.Integer,
 'deathCity': sqlalchemy.sql.sqltypes.String,
 'deathCountry': sqlalchemy.sql.sqltypes.String,
 'deathDay': sqlalchemy.sql.sqltypes.Integer,
 'deathMonth': sqlalchemy.sql.sqltypes.Integer,
 'deathState': sqlalchemy.sql.sqltypes.String,
 'deathYear': sqlalchemy.sql.sqltypes.Integer,
 'debut': sqlalchemy.sql.sqltypes.DateTime,
 'finalGame': sqlalchemy.sql.sqltypes.DateTime,
 'height': sqlalchemy.sql.sqltypes.Float,
 'nameFirst': sqlalchemy.sql.sqltypes.String,
 'nameGiven': sqlalchemy.sql.sqltypes.String,
 'nameLast': sqlalchemy.sql.sqltypes.String,
 'playerID': sqlalchemy.sql.sqltypes.String,
 'retroID': sqlalchemy.sql.sqltyp

In [59]:
from toolz import get
{col:get(dtype.kind, DTYPES_TO_SQLALCHEMY_TYPES) for dtype, col in zip(people.dtypes, people.columns)}

{'bats': sqlalchemy.sql.sqltypes.String,
 'bbrefID': sqlalchemy.sql.sqltypes.String,
 'birthCity': sqlalchemy.sql.sqltypes.String,
 'birthCountry': sqlalchemy.sql.sqltypes.String,
 'birthDay': sqlalchemy.sql.sqltypes.Integer,
 'birthMonth': sqlalchemy.sql.sqltypes.Integer,
 'birthState': sqlalchemy.sql.sqltypes.String,
 'birthYear': sqlalchemy.sql.sqltypes.Integer,
 'deathCity': sqlalchemy.sql.sqltypes.String,
 'deathCountry': sqlalchemy.sql.sqltypes.String,
 'deathDay': sqlalchemy.sql.sqltypes.Integer,
 'deathMonth': sqlalchemy.sql.sqltypes.Integer,
 'deathState': sqlalchemy.sql.sqltypes.String,
 'deathYear': sqlalchemy.sql.sqltypes.Integer,
 'debut': sqlalchemy.sql.sqltypes.DateTime,
 'finalGame': sqlalchemy.sql.sqltypes.DateTime,
 'height': sqlalchemy.sql.sqltypes.Float,
 'nameFirst': sqlalchemy.sql.sqltypes.String,
 'nameGiven': sqlalchemy.sql.sqltypes.String,
 'nameLast': sqlalchemy.sql.sqltypes.String,
 'playerID': sqlalchemy.sql.sqltypes.String,
 'retroID': sqlalchemy.sql.sqltyp

## <font color="red"> Exercise 4 </font>

Package your expression in a `lambda` and refactor your code by adding helper functions to clean up the expression.

In [60]:
# lambda function before refactoring
get_sql_types = lambda df: {col:DTYPES_TO_SQLALCHEMY_TYPES[dtype.kind] 
                            for col, dtype in zip(people.columns, people,dytpes)}

In [64]:
# refactored lambda function and helper functions
def get_sql_types(df):
    sql_type = lambda dtype: DTYPES_TO_SQLALCHEMY_TYPES[dtype.kind]
    cols_and_dtypes = lambda df: zip(df.columns, df.dtypes)
    return {col:sql_type(dtype) for col, dtype in cols_and_dtypes(df)}

In [65]:
get_sql_types(people)

{'bats': sqlalchemy.sql.sqltypes.String,
 'bbrefID': sqlalchemy.sql.sqltypes.String,
 'birthCity': sqlalchemy.sql.sqltypes.String,
 'birthCountry': sqlalchemy.sql.sqltypes.String,
 'birthDay': sqlalchemy.sql.sqltypes.Integer,
 'birthMonth': sqlalchemy.sql.sqltypes.Integer,
 'birthState': sqlalchemy.sql.sqltypes.String,
 'birthYear': sqlalchemy.sql.sqltypes.Integer,
 'deathCity': sqlalchemy.sql.sqltypes.String,
 'deathCountry': sqlalchemy.sql.sqltypes.String,
 'deathDay': sqlalchemy.sql.sqltypes.Integer,
 'deathMonth': sqlalchemy.sql.sqltypes.Integer,
 'deathState': sqlalchemy.sql.sqltypes.String,
 'deathYear': sqlalchemy.sql.sqltypes.Integer,
 'debut': sqlalchemy.sql.sqltypes.DateTime,
 'finalGame': sqlalchemy.sql.sqltypes.DateTime,
 'height': sqlalchemy.sql.sqltypes.Float,
 'nameFirst': sqlalchemy.sql.sqltypes.String,
 'nameGiven': sqlalchemy.sql.sqltypes.String,
 'nameLast': sqlalchemy.sql.sqltypes.String,
 'playerID': sqlalchemy.sql.sqltypes.String,
 'retroID': sqlalchemy.sql.sqltyp

## <font color="red"> Exercise 5 </font>

Add the `People.csv` to your `baseball.db`

In [ ]:
# lambda function before refactoring

In [1]:
# refactored lambda function and helper functions

## <font color="red"> Exercise 6 </font>

Set up a similar automatic conversion precess for `pyspark.DataFrames`. You need to build a structure like this:

```python
schema = StructType([StructField('Name', StringType(), True),
                     StructField('DateTime', TimestampType(), True)
                     StructField('Age', IntegerType(), True)])
```